### Матричные факторизации

В данной работе вам предстоит познакомиться с практической стороной матричных разложений.
Работа поделена на 4 задания:
1. Вам необходимо реализовать SVD разложения используя SGD на explicit данных
2. Вам необходимо реализовать матричное разложения используя ALS на implicit данных
3. Вам необходимо реализовать матричное разложения используя BPR на implicit данных
4. Вам необходимо реализовать матричное разложения используя WARP на implicit данных

Мягкий дедлайн 13 Октября (пишутся замечания, выставляется оценка, есть возможность исправить до жесткого дедлайна)

Жесткий дедлайн 20 Октября (Итоговая проверка)

In [1]:
import implicit
import pandas as pd
import numpy as np
import scipy.sparse as sp

from abc import ABC, abstractmethod
from typing import Tuple, List, Set, Dict
from lightfm.datasets import fetch_movielens
%load_ext autoreload
%autoreload 2

/opt/anaconda3/envs/recommendation-systems/lib/python3.8/site-packages/lightfm/_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn(


В данной работе мы будем работать с explicit датасетом movieLens, в котором представленны пары user_id movie_id и rating выставленный пользователем фильму

Скачать датасет можно по ссылке https://grouplens.org/datasets/movielens/1m/

In [2]:
ratings = pd.read_csv('./data/ratings.dat', delimiter='::', header=None, 
        names=['user_id', 'movie_id', 'rating', 'timestamp'], 
        usecols=['user_id', 'movie_id', 'rating'], engine='python')

In [3]:
movie_info = pd.read_csv('./data/movies.dat', delimiter='::', header=None, 
        names=['movie_id', 'name', 'category'], encoding="utf16", engine='python')

Explicit данные

In [4]:
ratings.head(10)

,user_id,movie_id,rating
0,1,1193,5
1,1,661,3
2,1,914,3
3,1,3408,4
4,1,2355,5
5,1,1197,3
6,1,1287,5
7,1,2804,5
8,1,594,4
9,1,919,4


Для того, чтобы преобразовать текущий датасет в Implicit, давайте считать что позитивная оценка это оценка >=4

In [5]:
implicit_ratings = ratings.loc[(ratings['rating'] >= 4)]

In [6]:
implicit_ratings.head(10)

,user_id,movie_id,rating
0,1,1193,5
3,1,3408,4
4,1,2355,5
6,1,1287,5
7,1,2804,5
8,1,594,4
9,1,919,4
10,1,595,5
11,1,938,4
12,1,2398,4


Удобнее работать с sparse матричками, давайте преобразуем DataFrame в CSR матрицы

In [7]:
users = implicit_ratings["user_id"]
movies = implicit_ratings["movie_id"]
user_item = sp.coo_matrix((np.ones_like(users), (users, movies)))
user_item_t_csr = user_item.T.tocsr()
user_item_csr = user_item.tocsr()

В качестве примера воспользуемся ALS разложением из библиотеки implicit

Зададим размерность латентного пространства равным 64, это же определяет размер user/item эмбедингов

In [8]:
model = implicit.als.AlternatingLeastSquares(factors=64, iterations=100, calculate_training_loss=True)

В качестве loss здесь всеми любимый RMSE

In [9]:
model.fit(user_item_t_csr)

  0%|          | 0/100 [00:00<?, ?it/s]

Построим похожие фильмы по 1 movie_id = Истории игрушек

In [10]:
movie_info.head(5)

,movie_id,name,category
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [11]:
get_similars = lambda item_id, model : [movie_info[movie_info["movie_id"] == x[0]]["name"].to_string() 
                                        for x in model.similar_items(item_id)]

Как мы видим, симилары действительно оказались симиларами.

Качество симиларов часто является хорошим способом проверить качество алгоритмов.

P.S. Если хочется поглубже разобраться в том как разные алгоритмы формируют разные латентные пространства, рекомендую загружать полученные вектора в tensorBoard и смотреть на сформированное пространство

In [12]:
get_similars(1, model)

['0    Toy Story (1995)',
 '3045    Toy Story 2 (1999)',
 "2286    Bug's Life, A (1998)",
 '33    Babe (1995)',
 '584    Aladdin (1992)',
 '2315    Babe: Pig in the City (1998)',
 '360    Lion King, The (1994)',
 '1526    Hercules (1997)',
 '1838    Mulan (1998)',
 '2618    Tarzan (1999)']

Давайте теперь построим рекомендации для юзеров

Как мы видим юзеру нравится фантастика, значит и в рекомендациях ожидаем увидеть фантастику

In [13]:
get_user_history = lambda user_id, implicit_ratings : [movie_info[movie_info["movie_id"] == x]["name"].to_string() 
                                            for x in implicit_ratings[implicit_ratings["user_id"] == user_id]["movie_id"]]

In [14]:
get_user_history(4, implicit_ratings)

['3399    Hustler, The (1961)',
 '2882    Fistful of Dollars, A (1964)',
 '1196    Alien (1979)',
 '1023    Die Hard (1988)',
 '257    Star Wars: Episode IV - A New Hope (1977)',
 '1959    Saving Private Ryan (1998)',
 '476    Jurassic Park (1993)',
 '1180    Raiders of the Lost Ark (1981)',
 '1885    Rocky (1976)',
 '1081    E.T. the Extra-Terrestrial (1982)',
 '3349    Thelma & Louise (1991)',
 '3633    Mad Max (1979)',
 '2297    King Kong (1933)',
 '1366    Jaws (1975)',
 '1183    Good, The Bad and The Ugly, The (1966)',
 '2623    Run Lola Run (Lola rennt) (1998)',
 '2878    Goldfinger (1964)',
 '1220    Terminator, The (1984)']

Получилось! 

Мы действительно порекомендовали пользователю фантастику и боевики, более того встречаются продолжения тех фильмов, которые он высоко оценил

In [15]:
get_recommendations = lambda user_id, model : [movie_info[movie_info["movie_id"] == x[0]]["name"].to_string() 
                                               for x in model.recommend(user_id, user_item_csr)]

In [16]:
get_recommendations(4, model)

['585    Terminator 2: Judgment Day (1991)',
 '1271    Indiana Jones and the Last Crusade (1989)',
 '1182    Aliens (1986)',
 '2502    Matrix, The (1999)',
 '1284    Butch Cassidy and the Sundance Kid (1969)',
 '1178    Star Wars: Episode V - The Empire Strikes Back...',
 '3402    Close Encounters of the Third Kind (1977)',
 '847    Godfather, The (1972)',
 '2460    Planet of the Apes (1968)',
 '453    Fugitive, The (1993)']

Теперь ваша очередь реализовать самые популярные алгоритмы матричных разложений

Что будет оцениваться:
1. Корректность алгоритма
2. Качество получившихся симиларов
3. Качество итоговых рекомендаций для юзера

In [17]:
import random 
from utils import get_rating, get_recommendations, get_similar_items
from models.sgd_model import SGDModel
from models.als_model import ALSModel
from models.bpr_model import BPRModel
from models.warp_model import WARPModel
from base import FactorizationModel

In [18]:
def set_seed(seed: int = 42): 
    random.seed(seed)
    np.random.seed(seed)

In [19]:
id2item = movie_info[["movie_id", "name"]].set_index("movie_id").to_dict()["name"]
id2genres = movie_info[["movie_id", "category"]].set_index("movie_id").to_dict()["category"]

In [20]:
def pretty_print(candidates_ids: List[int]): 
    for i, candidate_id in enumerate(candidates_ids, start=1): 
        if candidate_id not in id2item: 
            print(f"{i}. Item {candidate_id} not found.")
        else:
            print(f"{i}. {id2item[candidate_id]} ({id2genres[candidate_id]})")

In [21]:
def print_similars(item_id: int, model: FactorizationModel):
    similar_items_idx = get_similar_items(item_id, model, top_k=10)
    print(f"Similar items for '{id2item[item_id]}' ({id2genres[item_id]}):")
    pretty_print(similar_items_idx)

In [22]:
def print_recommendations(user_id: int, model: FactorizationModel): 
    recommend_items_idx = get_recommendations(user_id, model, top_k=10)
    print(f"Recommendations for '{user_id}':")
    pretty_print(recommend_items_idx)

### Задание 1. Не использую готовые решения, реализовать SVD разложение используя SGD на explicit данных

Создадим explicit набор данных и обучим модель. 

In [23]:
set_seed()
user_item_explicit = sp.coo_matrix((ratings["rating"], (ratings["user_id"], ratings["movie_id"]))).tocsr()
sgd_model = SGDModel(factors=64, lr=1e-2, iterations=50, verbose=True, verbose_every=10)
sgd_model = sgd_model.fit(user_item_explicit)

 20%|██        | 10/50 [01:40<06:49, 10.24s/it]

Iter: 10 | RMSE: 0.68 | Elapsed: 1m40s


 40%|████      | 20/50 [03:09<04:22,  8.76s/it]

Iter: 20 | RMSE: 0.58 | Elapsed: 3m10s


 60%|██████    | 30/50 [04:36<02:52,  8.65s/it]

Iter: 30 | RMSE: 0.54 | Elapsed: 4m36s


 80%|████████  | 40/50 [06:02<01:27,  8.72s/it]

Iter: 40 | RMSE: 0.53 | Elapsed: 6m2s


100%|██████████| 50/50 [07:29<00:00,  9.00s/it]

Iter: 50 | RMSE: 0.53 | Elapsed: 7m30s


Проверим качество обученной модели.

In [24]:
print_similars(1, sgd_model)

Similar items for 'Toy Story (1995)' (Animation|Children's|Comedy):
1. Toy Story (1995) (Animation|Children's|Comedy)
2. Toy Story 2 (1999) (Animation|Children's|Comedy)
3. Bug's Life, A (1998) (Animation|Children's|Comedy)
4. Wrong Trousers, The (1993) (Animation|Comedy)
5. Close Shave, A (1995) (Animation|Comedy|Thriller)
6. Aladdin (1992) (Animation|Children's|Comedy|Musical)
7. Back to the Future (1985) (Comedy|Sci-Fi)
8. Grand Day Out, A (1992) (Animation|Comedy)
9. My Man Godfrey (1936) (Comedy)
10. Shawshank Redemption, The (1994) (Drama)


Как можно видеть, фильмы достаточно подходящие.

In [25]:
print_recommendations(4, sgd_model)

Recommendations for '4':
1. Casablanca (1942) (Drama|Romance|War)
2. Lawrence of Arabia (1962) (Adventure|War)
3. Shadow of a Doubt (1943) (Film-Noir|Thriller)
4. Face/Off (1997) (Action|Sci-Fi|Thriller)
5. Animal House (1978) (Comedy)
6. Bicycle Thief, The (Ladri di biciclette) (1948) (Drama)
7. Mr. Holland's Opus (1995) (Drama)
8. Silence of the Lambs, The (1991) (Drama|Thriller)
9. Boat, The (Das Boot) (1981) (Action|Drama|War)
10. It's a Wonderful Life (1946) (Drama)


Фантастики и боевиков получилось мало :( Пробуем более крутые методы. 

Видимо, придется к хард-дедлайну делать SGD со смещениями и регуляризацией и мини-батчами...

### Задание 2. Не использую готовые решения, реализовать матричное разложение используя ALS на implicit данных

In [26]:
set_seed()
user_item_implicit = user_item_csr
als_model = ALSModel(factors=64, iterations=10, lambd=1e-5, verbose=True, verbose_every=1)
als_model = als_model.fit(user_item_implicit)

 10%|█         | 1/10 [00:14<02:11, 14.61s/it]

Iter: 1 | RMSE: 0.73 | Elapsed: 0m14s


 20%|██        | 2/10 [00:28<01:54, 14.28s/it]

Iter: 2 | RMSE: 0.65 | Elapsed: 0m28s


 30%|███       | 3/10 [00:42<01:38, 14.04s/it]

...

Iter: 8 | RMSE: 0.64 | Elapsed: 1m52s


 90%|█████████ | 9/10 [02:07<00:14, 14.21s/it]

Iter: 9 | RMSE: 0.64 | Elapsed: 2m7s


100%|██████████| 10/10 [02:23<00:00, 14.38s/it]

Iter: 10 | RMSE: 0.64 | Elapsed: 2m23s


Проверим качество обученное модели. 

In [27]:
print_similars(1, als_model)

Similar items for 'Toy Story (1995)' (Animation|Children's|Comedy):
1. Toy Story (1995) (Animation|Children's|Comedy)
2. Toy Story 2 (1999) (Animation|Children's|Comedy)
3. Bug's Life, A (1998) (Animation|Children's|Comedy)
4. Babe (1995) (Children's|Comedy|Drama)
5. Aladdin (1992) (Animation|Children's|Comedy|Musical)
6. Hercules (1997) (Adventure|Animation|Children's|Comedy|Musical)
7. Mulan (1998) (Animation|Children's)
8. Babe: Pig in the City (1998) (Children's|Comedy)
9. Lion King, The (1994) (Animation|Children's|Musical)
10. Tarzan (1999) (Animation|Children's)


Почти все фильмы оказались подходящими!

In [28]:
print_recommendations(4, als_model)

Recommendations for '4':
1. Raiders of the Lost Ark (1981) (Action|Adventure)
2. Star Wars: Episode IV - A New Hope (1977) (Action|Adventure|Fantasy|Sci-Fi)
3. Jaws (1975) (Action|Horror)
4. Terminator, The (1984) (Action|Sci-Fi|Thriller)
5. Saving Private Ryan (1998) (Action|Drama|War)
6. E.T. the Extra-Terrestrial (1982) (Children's|Drama|Fantasy|Sci-Fi)
7. Alien (1979) (Action|Horror|Sci-Fi|Thriller)
8. Die Hard (1988) (Action|Thriller)
9. Terminator 2: Judgment Day (1991) (Action|Sci-Fi|Thriller)
10. Goldfinger (1964) (Action)


Вот, здесь все норм. 

### Задание 3. Не использую готовые решения, реализовать матричное разложение BPR на implicit данных

In [29]:
from models.bpr_model import BPRModel

In [30]:
set_seed()
user_item_implicit = user_item_csr
bpr_model = BPRModel(factors=64, lr=0.1, iterations=10, lambd=1e-6, verbose=True, verbose_every=1)
bpr_model = bpr_model.fit(user_item_implicit)

 10%|█         | 1/10 [00:40<06:07, 40.86s/it]

Iter: 1 | Triplet acc: 0.88 | Elapsed: 0m40s


 20%|██        | 2/10 [01:18<05:09, 38.72s/it]

Iter: 2 | Triplet acc: 0.89 | Elapsed: 1m18s


 30%|███       | 3/10 [01:59<04:40, 40.13s/it]

...

Iter: 8 | Triplet acc: 0.95 | Elapsed: 5m16s


 90%|█████████ | 9/10 [05:54<00:39, 39.35s/it]

Iter: 9 | Triplet acc: 0.95 | Elapsed: 5m54s


100%|██████████| 10/10 [06:32<00:00, 39.25s/it]

Iter: 10 | Triplet acc: 0.95 | Elapsed: 6m32s


In [31]:
print_similars(1, bpr_model)

Similar items for 'Toy Story (1995)' (Animation|Children's|Comedy):
1. Toy Story (1995) (Animation|Children's|Comedy)
2. Toy Story 2 (1999) (Animation|Children's|Comedy)
3. Star Wars: Episode IV - A New Hope (1977) (Action|Adventure|Fantasy|Sci-Fi)
4. Back to the Future (1985) (Comedy|Sci-Fi)
5. Bug's Life, A (1998) (Animation|Children's|Comedy)
6. Terminator 2: Judgment Day (1991) (Action|Sci-Fi|Thriller)
7. Babe (1995) (Children's|Comedy|Drama)
8. Aladdin (1992) (Animation|Children's|Comedy|Musical)
9. Groundhog Day (1993) (Comedy|Romance)
10. Fugitive, The (1993) (Action|Thriller)


In [32]:
print_recommendations(4, bpr_model)

Recommendations for '4':
1. Raiders of the Lost Ark (1981) (Action|Adventure)
2. Star Wars: Episode IV - A New Hope (1977) (Action|Adventure|Fantasy|Sci-Fi)
3. Star Wars: Episode V - The Empire Strikes Back (1980) (Action|Adventure|Drama|Sci-Fi|War)
4. Terminator 2: Judgment Day (1991) (Action|Sci-Fi|Thriller)
5. Godfather: Part II, The (1974) (Action|Crime|Drama)
6. Godfather, The (1972) (Action|Crime|Drama)
7. Jaws (1975) (Action|Horror)
8. Matrix, The (1999) (Action|Sci-Fi|Thriller)
9. Saving Private Ryan (1998) (Action|Drama|War)
10. Jurassic Park (1993) (Action|Adventure|Sci-Fi)


**Great! Fantastic! BPR is all you need!**

### Задание 4. Не использую готовые решения, реализовать матричное разложение WARP на implicit данных

In [34]:
set_seed()
user_item_implicit = user_item_csr
warp_model = WARPModel(factors=64, lr=1e-2, iterations=1, verbose=True, verbose_every=1)
warp_model = warp_model.fit(user_item_implicit)

100%|██████████| 1/1 [16:22<00:00, 982.47s/it]

Iter: 1 | Triplet acc: 0.99 | Elapsed: 16m22s


In [35]:
print_similars(1, warp_model)

Similar items for 'Toy Story (1995)' (Animation|Children's|Comedy):
1. Toy Story (1995) (Animation|Children's|Comedy)
2. Designated Mourner, The (1997) (Drama)
3. Matilda (1996) (Children's|Comedy)
4. Scrooged (1988) (Comedy)
5. Better Than Chocolate (1999) (Comedy|Romance)
6. Oliver & Company (1988) (Animation|Children's)
7. Borrowers, The (1997) (Adventure|Children's|Comedy|Fantasy)
8. Coogan's Bluff (1968) (Crime)
9. Jungle Book, The (1994) (Adventure|Children's|Romance)
10. White Men Can't Jump (1992) (Comedy)


In [36]:
print_recommendations(4, warp_model)

Recommendations for '4':
1. Jurassic Park (1993) (Action|Adventure|Sci-Fi)
2. Jaws (1975) (Action|Horror)
3. Braveheart (1995) (Action|Drama|War)
4. Raiders of the Lost Ark (1981) (Action|Adventure)
5. Saving Private Ryan (1998) (Action|Drama|War)
6. Alien (1979) (Action|Horror|Sci-Fi|Thriller)
7. Men in Black (1997) (Action|Adventure|Comedy|Sci-Fi)
8. Indiana Jones and the Last Crusade (1989) (Action|Adventure)
9. Terminator, The (1984) (Action|Sci-Fi|Thriller)
10. E.T. the Extra-Terrestrial (1982) (Children's|Drama|Fantasy|Sci-Fi)


Вроде неплохо, что-то релевантное порекомендовалось. 